In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.3.6
# 二叉搜索树中最接近的值 II：lintcode 901

In [10]:
## 使用两个Iterator，把二叉树看成数组，一个Iterator向更大的值移动，也就是next操作
## 另一个Iterator向更小的值移动，也就是prev操作
class Solution:
    """
    @param root: the given BST
    @param target: the given target
    @param k: the given k
    @return: k values in the BST that are closest to the target
    """
    def closestKValues(self, root, target, k):
        # write your code here
        ## 下边界栈和上边界栈初始时是相同的
        ## 它们的栈顶元素都是最接近target的值
        lower_stack = self.get_stack(root, target)
        upper_stack = list(lower_stack)
        
        if lower_stack[-1].val < target:
            ## 如果最接近target的结点值小于target
            ## 将栈顶元素按中序遍历的下一个结点压入栈中，这个结点肯定大于等于target
            ## 否则之前在get_stack时就会把它放到栈顶元素
            self.move_upper(upper_stack)
        else:
            ## 同理，如果target小于栈顶元素，则将栈顶元素的上一个结点压入栈中，这个结点肯定小于target
            self.move_lower(lower_stack)
        
        result = []
        ## 类比二分法中求最接近的k个值
        for i in range(k):
            if self.is_lower_closet(lower_stack, upper_stack, target):
                result.append(lower_stack[-1].val)
                self.move_lower(lower_stack)
            else:
                result.append(upper_stack[-1].val)
                self.move_upper(upper_stack)
        
        return result
            
        
    ## 找到树中最接近target的结点，并把沿途的结点都入栈
    def get_stack(self, root, target):
        stack = []
        ## 如果target小于根结点值，root就向左子树移动
        ## 反之，则向右子树移动，当root为空时，stack栈顶的结点
        ## 就是最接近target的结点，有可能比target大，也可能比target小
        while root:
            stack.append(root)
            if target < root.val:
                root = root.left
            else:
                root = root.right
        return stack
    
    def is_lower_closet(self, lower_stack, upper_stack, target):
        if not lower_stack:
            return False
        if not upper_stack:
            return True
        
        ## 实际上lower_stack中存储的有比target大的值，但其栈顶元素肯定比target小
        ## 执行一次move_lower时，会把那些大于target的结点都弹出栈，因为move_lower
        ## 是查找栈顶元素按中序遍历的前一个元素，只要初始时栈顶元素的值小于target
        ## 后续的栈顶元素肯定比target小
        return target - lower_stack[-1].val < upper_stack[-1].val - target
    
    ## 相当于lintcode 86中BST迭代器中的next操作
    ## 将原stack栈顶元素的按中序遍历的下一个结点以及沿途的结点压入栈中
    def move_upper(self, stack):
        node = stack[-1]
        if node.right:
            node = node.right
            while node:
                stack.append(node)
                node = node.left
        else:
            node = stack.pop()
            while stack and stack[-1].right == node:
                node = stack.pop()
    
    ## 实现一个prev操作，找到栈顶元素按中序遍历的前一个结点
    ## 是next操作的镜像
    def move_lower(self, stack):
        node = stack[-1]
        if node.left:
            node = node.left
            while node:
                stack.append(node)
                node = node.right
        else:
            node = stack.pop()
            while stack and stack[-1].left == node:
                node = stack.pop()
